In [ ]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from keras.applications.densenet import DenseNet169
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images_path = {}
images_path["Covid"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.png")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpeg")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpg")

images_path["NoFindings"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.png")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpeg")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpg")

images_path["Pneumonia"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.png")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.jpeg")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.jpg")
images_class = {
    "Covid": 0,
    "Pneumonia": 1,
    "NoFindings": 2
}

In [ ]:
X = []
Y = []

for label in images_path:
    for image_path in images_path[label]:
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224, 224))
        X.append(image)
        Y.append(images_class[label])

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, Y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train=Y[train_index]
    y_test=Y[test_index]

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3).astype('float32')
y_train=np.array(y_train)
X_test=np.array(X_test).reshape(-1,224,224,3).astype('float32')
y_test=np.array(y_test)

In [ ]:
X_train=tf.keras.applications.densenet.preprocess_input(X_train)
X_test=tf.keras.applications.densenet.preprocess_input(X_test)

In [ ]:
densenet.summary()

In [ ]:
densenet=tf.keras.applications.DenseNet169(input_shape=(224,224,3),weights='imagenet',include_top=False)

51888128/51877672 [==============================] - 0s 0us/step


In [ ]:
densenet.trainable = False
for layer in densenet.layers:
     layer.trainable = False

In [ ]:
base_model_output = densenet.output
gap = keras.layers.GlobalAveragePooling2D()(base_model_output)
fc3=keras.layers.Dense(units=512, activation="relu",kernel_regularizer='l2')(gap)
BN3=tf.keras.layers.BatchNormalization()(fc3)
output = keras.layers.Dense(3, activation="softmax")(BN3)

In [ ]:
transfer_model = keras.models.Model(densenet.input, output)

In [ ]:
transfer_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [ ]:
transfer_model.compile(loss="sparse_categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,epochs=40,batch_size=8,validation_split=0.1)

Epoch 1/40
102/102 [==============================] - 25s 61ms/step - loss: 4.0072 - accuracy: 0.6802 - val_loss: 3.6466 - val_accuracy: 0.5556
Epoch 2/40
102/102 [==============================] - 4s 35ms/step - loss: 1.9469 - accuracy: 0.8296 - val_loss: 2.5127 - val_accuracy: 0.6444
Epoch 3/40
102/102 [==============================] - 4s 35ms/step - loss: 1.3972 - accuracy: 0.8395 - val_loss: 1.6219 - val_accuracy: 0.6778
Epoch 4/40
102/102 [==============================] - 4s 35ms/step - loss: 1.1077 - accuracy: 0.8568 - val_loss: 1.4992 - val_accuracy: 0.6889
Epoch 5/40
102/102 [==============================] - 4s 35ms/step - loss: 0.9012 - accuracy: 0.8728 - val_loss: 1.1163 - val_accuracy: 0.6778
Epoch 6/40
102/102 [==============================] - 4s 35ms/step - loss: 0.7893 - accuracy: 0.8691 - val_loss: 2.8784 - val_accuracy: 0.5111
Epoch 7/40
102/102 [==============================] - 4s 35ms/step - loss: 0.7774 - accuracy: 0.8469 - val_loss: 1.9260 - val_accuracy: 0.588

In [ ]:
densenet.trainable = True
for layer in densenet.layers:
     layer.trainable = True

In [ ]:
ADAM= keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(loss="sparse_categorical_crossentropy",optimizer=ADAM,metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,batch_size=8,epochs=100,validation_split=0.1)

Epoch 1/100
102/102 [==============================] - 28s 124ms/step - loss: 1.0471 - accuracy: 0.7284 - val_loss: 0.8634 - val_accuracy: 0.7889
Epoch 2/100
102/102 [==============================] - 9s 92ms/step - loss: 0.4806 - accuracy: 0.8691 - val_loss: 0.7667 - val_accuracy: 0.7444
Epoch 3/100
102/102 [==============================] - 9s 92ms/step - loss: 0.3241 - accuracy: 0.9259 - val_loss: 0.8460 - val_accuracy: 0.7333
Epoch 4/100
102/102 [==============================] - 9s 92ms/step - loss: 0.3705 - accuracy: 0.9086 - val_loss: 0.7838 - val_accuracy: 0.7000
Epoch 5/100
102/102 [==============================] - 9s 92ms/step - loss: 0.2829 - accuracy: 0.9395 - val_loss: 0.7516 - val_accuracy: 0.7444
Epoch 6/100
102/102 [==============================] - 9s 91ms/step - loss: 0.2663 - accuracy: 0.9593 - val_loss: 0.7754 - val_accuracy: 0.7556
Epoch 7/100
102/102 [==============================] - 9s 91ms/step - loss: 0.2357 - accuracy: 0.9568 - val_loss: 0.7713 - val_accurac

In [ ]:
mse_test = transfer_model.evaluate(X_test, y_test)

8/8 [==============================] - 2s 140ms/step - loss: 0.3782 - accuracy: 0.8800
